In [1]:
import socket
import time
import numpy as np

In [3]:
IP_ADDRESS = "192.168.0.17"
TCP_Port = 23
UDP_Port = 4660

In [26]:
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)

In [141]:
address = 0x189
readBytes = bytearray([0xFF,0xc0,0xaa,0x01]) + address.to_bytes(4,'big')
print(readBytes)

bytearray(b'\xff\xc0\xaa\x01\x00\x00\x01\x89')


In [126]:
bin(0x80)

'0b10000000'

In [143]:
address = 0x189
data = 0x40
writeBytes = bytearray([0xFF,0x80,0xaa,0x01]) + address.to_bytes(4,'big') + bytes([data])
print(writeBytes)

bytearray(b'\xff\x80\xaa\x01\x00\x00\x01\x89@')


In [145]:
sock.sendto(readBytes,(IP_ADDRESS,UDP_Port))
rec = sock.recv(1024)

print(f"Header: {rec[:8]}\nData: {rec[8:]}")

Header: b'\xff\xc8\xaa\x01\x00\x00\x01\x89'
Data: b'@'


In [48]:
len(rec[0])

16

In [21]:
rec[0]

b'\xff\xc8\x00\xff\x00\x00\x01\xb5\x00\x90\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xea`\x13\x88+\xf2\x01\xf4\x13\x87\t\x7f\x7f\x7f\x7f\x7f\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8'

In [12]:
bin(0x8)

'0b1000'

In [2]:
0x01

1

In [54]:
def UDP_Neunet(IP='192.168.0.17',port=4660,mode='r',ID=None,length=1,address=0x80,data=None,printOutput=True):
        if mode == 'r':
                modeByte = 0xc0
        elif mode == 'w':
                modeByte = 0x80
        if ID is None:
                ID = np.random.randint(0,255)
        sendBytes = bytearray([0xff,modeByte,ID,length]) + address.to_bytes(4,'big')
        if data is not None:
                sendBytes += bytes([data])
                
        with socket.socket(socket.AF_INET,socket.SOCK_DGRAM) as sock:
                sock.sendto(sendBytes,(IP,port))
                recvData = sock.recv(1024)
        if printOutput:
                print(f"Send: {sendBytes.hex(':')}\n",f"Header : {recvData[:8].hex(':')}\n",f"Data : {recvData[8:].hex(':')}")
        return recvData

In [60]:
UDP_Neunet(address=0x1b5,mode='r',length=1)

Send: ff:c0:66:01:00:00:01:b5
 Header : ff:c8:66:01:00:00:01:b5
 Data : 00


b'\xff\xc8f\x01\x00\x00\x01\xb5\x00'

In [50]:
bin((output[1] % 2**4) >> 3)

'0b1'

In [53]:
bin(0x80)

'0b10000000'